# spacetrackのテスト

see https://pythonhosted.org/spacetrack/usage.html

In [1]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pandas as pd
import json
import time

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 80)

In [3]:
# userid and password are set in here
import spacetrackaccount
# print("Login with " + spacetrackaccount.userid)

In [4]:
st = SpaceTrackClient(spacetrackaccount.userid, spacetrackaccount.password)

In [5]:
def mycallback(until):
    duration = int(round(until - time.monotonic()))
    print('Sleeping for {:d} seconds.'.format(duration))

st.callback = mycallback

In [6]:
# example
#data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30',
#                     mean_motion=op.inclusive_range(0.99, 1.01),
#                     eccentricity=op.less_than(0.01), format='tle')

In [7]:
# 全衛星の最新データ(30日以内)
data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='3le')

In [8]:
with open('download/tle_latest.txt', 'w') as fp:
    for line in data:
        fp.write(line + '\n')

In [9]:
# 特定の衛星の過去の全データ
data = st.tle(iter_lines=True, norad_cat_id=25544, orderby='epoch', format='tle')

In [10]:
with open('download/tle_25544_all.txt', 'w') as fp:
    for line in data:
        fp.write(line + '\n')

In [11]:
# 複数の衛星の過去の全データをjsonで取得 (まとめて取得するので、同時に多くの衛星を指定することはできない)
data = st.tle(norad_cat_id=[op.inclusive_range(1, 20)], orderby=['norad_cat_id', 'epoch'], format='json')

In [12]:
with open('download/test.json', 'w') as fp:
        fp.write(data)

In [13]:
df = pd.read_json(data, orient='records')

In [14]:
df.head(3)

,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-09 22:15:15,874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,999,3800,0.0,0.000806,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001,1958-001A,4,7615.485,110.231,2117.362,357.338,1
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-17 13:46:31,735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,999,3900,0.0,0.000830,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007,1958-001A,4,7610.615,110.125,2107.621,357.339,1
2,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-08-01 20:45:25,598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,999,4100,0.0,0.000632,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006,1958-001A,4,7611.820,110.151,2110.015,357.355,1


In [15]:
df.TLE_LINE1[0]

'1 00004U 58001  A 59190.92726707  .00080584 +00000-0 +00000-0 0  9997'

In [16]:
# 複数の衛星の過去の全データをjsonで取得しファイルに保存する (forで取得する)
for i in range(1, 20):
    print(i)
    data = st.tle(norad_cat_id=i, orderby=['norad_cat_id', 'epoch'], format='json')
    file = 'download/{}.json'.format(i)
    with open(file, 'w') as fp:
        fp.write(data)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [17]:
# 適当なファイルを読んでみる
df = pd.read_json('download/4.json', orient='records')
df.head(3)

,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-09 22:15:15,874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,999,3800,0,0.000806,0,33905,0 EXPLORER 1,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001,1958-001A,4,7615.485,110.231,2117.362,357.338,1
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-17 13:46:31,735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,999,3900,0,0.000830,0,33905,0 EXPLORER 1,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007,1958-001A,4,7610.615,110.125,2107.621,357.339,1
2,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-08-01 20:45:25,598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,999,4100,0,0.000632,0,33905,0 EXPLORER 1,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006,1958-001A,4,7611.820,110.151,2110.015,357.355,1


https://www.space-track.org/documentation#faq によると、現在では tle, tle_latest API は 非推奨なので、gp, gp_history API を試してみる。

gp APIで取得できるデータの項目: https://www.space-track.org/basicspacedata/modeldef/class/gp/format/html

In [18]:
# 全衛星の最新データ(30日以内)
data = st.gp(epoch='>now-30', orderby='norad_cat_id', format='json')

In [19]:
with open('download/gp_30day.json', 'w') as fp:
    fp.write(data)

In [20]:
df = pd.read_json(data)

In [21]:
# APOAPSIS, PERIAPSIS など項目名が変わったり、COUNTRY_CODE, LAUNCH_DATEなど項目が追加されたりしているが、基本的には上位互換
df.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2020-11-02T02:24:39,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-11-01T21:34:58.044288,10.848689,0.184706,34.2556,164.8429,250.9051,88.3214,0,U,5,999,22018,-0.000017,6.000000e-08,0.0,8619.531,132.734,3833.479,649.314,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,None,2868521,164658062,0 VANGUARD 1,1 5U 58002B 20306.89928292 +.00000006 +00000-0 -17189-4 0 9997,2 5 034.2556 164.8429 1847064 250.9051 088.3214 10.84868882220180
1,2,GENERATED VIA SPACE-TRACK.ORG API,2020-11-02T02:24:39,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-11-01T20:57:46.851552,11.856844,0.146677,32.8751,259.4579,307.2305,40.3058,0,U,11,999,29129,0.000108,2.270000e-06,0.0,8123.735,121.448,2937.166,554.034,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,None,2868521,164658150,0 VANGUARD 2,1 11U 59001A 20306.87345893 +.00000227 +00000-0 +10811-3 0 9990,2 11 032.8751 259.4579 1466771 307.2305 040.3058 11.85684445291292
2,2,GENERATED VIA SPACE-TRACK.ORG API,2020-11-02T02:24:39,18 SPCS,VANGUARD R/B,1959-001B,EARTH,TEME,UTC,SGP4,2020-11-01T22:47:03.134112,11.443462,0.166637,32.8977,229.9158,78.8198,299.4413,0,U,12,999,29573,0.000231,3.800000e-06,0.0,8318.216,125.836,3326.203,553.959,ROCKET BODY,MEDIUM,US,1959-02-17,AFETR,None,2868521,164658343,0 VANGUARD R/B,1 12U 59001B 20306.94934183 +.00000380 +00000-0 +23141-3 0 9995,2 12 032.8977 229.9158 1666369 078.8198 299.4413 11.44346170295734


In [22]:
# 過去のデータをjsonで取得
data = st.gp_history(norad_cat_id=4, orderby=['norad_cat_id', 'epoch'], format='json', limit=10)

In [23]:
with open('download/gp_history_4.json', 'w') as fp:
    fp.write(data)

In [24]:
df = pd.read_json(data)

In [25]:
# 古い衛星についてはOBJECT_NAME と TLE_LINE0 が TBA - TO BE ASSIGNED となってしまうことがある
df.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:35:20,18 SPCS,TBA - TO BE ASSIGNED,1958-001A,EARTH,TEME,UTC,SGP4,1959-07-09T22:15:15.874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,U,4,999,3800,0,0.000806,0,7615.485,110.231,2117.362,357.338,TBA,NaN,NaN,NaN,NaN,NaN,33905,8671417,0 TBA - TO BE ASSIGNED,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001
1,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:35:20,18 SPCS,TBA - TO BE ASSIGNED,1958-001A,EARTH,TEME,UTC,SGP4,1959-07-17T13:46:31.735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,U,4,999,3900,0,0.000830,0,7610.615,110.125,2107.621,357.339,TBA,NaN,NaN,NaN,NaN,NaN,33905,8671418,0 TBA - TO BE ASSIGNED,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007
2,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:35:20,18 SPCS,TBA - TO BE ASSIGNED,1958-001A,EARTH,TEME,UTC,SGP4,1959-08-01T20:45:25.598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,U,4,999,4100,0,0.000632,0,7611.820,110.151,2110.015,357.355,TBA,NaN,NaN,NaN,NaN,NaN,33905,8671419,0 TBA - TO BE ASSIGNED,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006


In [26]:
# EPOCHが特定の日の全データを取得 (creation_dateでもセレクション可)
data = st.gp_history(epoch=[op.inclusive_range('2020-01-01', '2020-01-02')], orderby=['norad_cat_id', 'epoch'], format='json')

In [27]:
with open('download/gp_history_2020-01-01.json', 'w') as fp:
    fp.write(data)

In [28]:
df = pd.read_json(data)

In [29]:
df.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2020-01-01T06:30:17,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-01-01T03:02:06.650304,10.847946,0.184560,34.2579,26.8849,314.8569,31.5834,0,U,5,999,18707,0.000245,0.000002,0.0,8619.925,132.744,3832.684,650.896,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,NaN,2676442,145897261,0 VANGUARD 1,1 5U 58002B 20001.12646586 .00000199 00000-0 24469-3 0 9998,2 5 34.2579 26.8849 1845601 314.8569 31.5834 10.84794552187077
1,2,GENERATED VIA SPACE-TRACK.ORG API,2020-01-01T22:40:17,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-01-01T19:17:00.265056,11.856477,0.146792,32.8693,324.8532,25.0587,341.4531,0,U,11,999,59681,0.000153,0.000003,0.0,8123.902,121.452,2938.295,553.239,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,NaN,2676872,145926903,0 VANGUARD 2,1 11U 59001A 20001.80347529 .00000282 00000-0 15330-3 0 9995,2 11 32.8693 324.8532 1467925 25.0587 341.4531 11.85647749596815
2,2,GENERATED VIA SPACE-TRACK.ORG API,2020-01-02T06:13:51,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-01-01T21:18:20.157984,11.856478,0.146792,32.8692,324.5369,25.5344,341.0943,0,U,11,999,25508,0.000155,0.000003,0.0,8123.902,121.452,2938.295,553.239,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,NaN,2676929,145928075,0 VANGUARD 2,1 11U 59001A 20001.88773331 +.00000284 +00000-0 +15457-3 0 9999,2 11 032.8692 324.5369 1467925 025.5344 341.0943 11.85647808255085
